# 0. [OPTIONAL] Installing course dependencies

These are dependencies for the whole course.

In [ ]:
!pip install -r ../requirements.txt

You may skip the next block for now. You will need `ffmpeg` on week 12.

In [ ]:
# !conda update -y base conda
!conda install -c conda-forge ffmpeg -y

Run the next cell if you want to download embedding model, but this is not required during this lab. You can do it later.

In [ ]:
!python -m spacy download en_trf_distilbertbaseuncased_lg

# 1. Touching the Internet

Solve the following task.
1. Download [this page](https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt)
2. Save it to the file with the **unique** name derived from the URL. NB File with another URL should not be save into the file with this name. E.g. [this file](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/facts.txt) is another file with another content!

Hints:
- [requests](https://docs.python-requests.org/en/latest/) library is cool.
- [hashlib](https://docs.python.org/3/library/hashlib.html) may help with computing hash strings.
- when you download and save the data, don't try to encode and decode it. Use binary format when working with streams and files. <span style="color:red">Discuss with your TA which encodings you know and how they differ</span>.

In [2]:
import requests
import hashlib
url1 = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
url2 = "https://github.com/IUCVLab/information-retrieval/blob/main/datasets/facts.txt"

r = requests.get(url1)
open(hashlib.sha256(url1.encode()).hexdigest()+ ".txt", "w",  encoding="utf-8").write(r.text)

r = requests.get(url2)
open(hashlib.sha256(url2.encode()).hexdigest()+ ".txt", "w",  encoding="utf-8" ).write(r.text)

193203

# 2. Parsing different formats

Most probably, if you meet something in the Internet, this is one of: binary, plain text, XML, or json. XML then splits into xHTML, RSS, Atom, SOAP, XML-RPC, ... . Your task is to learn, how to process different formats.

## 2.1. JSON

In [the given file](http://sprotasov.ru/data/postnauka.txt) there is valid json. Parse this file and print all video URLs, which have `computer science` tag. Use built-in features of `requests`, or just a `json` library ([ref](https://docs.python.org/3/library/json.html)).

Hint:
- if the file has issues with parsing read about [the difference](https://stackoverflow.com/questions/57152985/what-is-the-difference-between-utf-8-and-utf-8-sig).

In [15]:
import json
import requests

url = "http://sprotasov.ru/data/postnauka.txt"
r = requests.get(url)

r.encoding = "UTF-8-sig"

content = r.json()

for dic in content:
        if "computer science" in dic["tags"]:
                print(dic["url"])

http://postnauka.ru/talks/31897
http://postnauka.ru/video/24306
http://postnauka.ru/faq/46974


## 2.2. HTML

For a given StackExchange answer extract logins of the contributors (who asked and who answered) with votes. [bs4](https://beautiful-soup-4.readthedocs.io/en/latest/) will help you to do the job.

I can recommend to use CSS or XPath selectors. `div` elements with `post-layout` class represent answers. Inside there are `div` with `votecell` class stroring votes number and `div` with class `user-details` storing user info. My personal recommendation is to use `css selectors`, which are [documented here](https://beautiful-soup-4.readthedocs.io/en/latest/#css-selectors).

In [32]:
import requests
from bs4 import BeautifulSoup

url = "https://math.stackexchange.com/questions/411486/"\
        "understanding-the-singular-value-decomposition-svd"


# TODO. Your code here should parse HTML source page and find contributors of the repository.

r = requests.get(url)
html_doc = r.text

soup = BeautifulSoup(html_doc, "html.parser")
soup.select(".user-details")

[<div class="user-details">
 <a href="/users/339790/rodrigo-de-azevedo">Rodrigo de Azevedo</a>
 <div class="-flair">
 <span class="reputation-score" dir="ltr" title="reputation score 19,568">19.6k</span><span aria-hidden="true" title="5 gold badges"><span class="badge1"></span><span class="badgecount">5</span></span><span class="v-visible-sr">5 gold badges</span><span aria-hidden="true" title="38 silver badges"><span class="badge2"></span><span class="badgecount">38</span></span><span class="v-visible-sr">38 silver badges</span><span aria-hidden="true" title="99 bronze badges"><span class="badge3"></span><span class="badgecount">99</span></span><span class="v-visible-sr">99 bronze badges</span>
 </div>
 </div>,
 <div class="user-details" itemprop="author" itemscope="" itemtype="http://schema.org/Person">
 <a href="/users/81007/celdor">Celdor</a><span class="d-none" itemprop="name">Celdor</span>
 <div class="-flair">
 <span class="reputation-score" dir="ltr" title="reputation score ">61

# 2.3. RSS feed

A lot of information is already organized in typed XML documents. Podcasts, for example, are just RSS feed. Parse [the feed of this podcast](http://sprotasov.ru/podcast/rss.xml) and print out:
- the number of episodes
- the length of the time span between the first and the last episodes (in days).

Use [`feedparser` library for this](https://waylonwalker.com/parsing-rss-python/).

In [39]:
import feedparser
rss = 'http://sprotasov.ru/podcast/rss.xml'
# feedparser.parse(rss)

# TODO: complete the code to compute the time span of all the episodes.

feed= feedparser.parse(rss)
print(feed.keys())
print(f"Number of episodes is {len(feed['entries'])}")

print(feed["entries"][0]["published"])
print(feed["entries"][-1]["published"])

dict_keys(['bozo', 'entries', 'feed', 'headers', 'etag', 'updated', 'updated_parsed', 'href', 'status', 'encoding', 'version', 'namespaces'])
Number of episodes is 18
Mon, 14 November 2022 21:25:00 +0300
Mon, 24 Apr 2017 12:00:00 +0300


# 3. [EXTRA TASK] Solving simple information retrieval task

According to the name, `information retrieval` is the discipline, which helps retrieves information (from unstructured sources). Thus, we will retrieve some information from [this news article](https://www.bbc.com/news/world-us-canada-59944889). Your task is to write a code, which will answer the question: **How many people die every day in the US waiting for a transplant?** Write flexible enough code. Test yourself by changing the link to [this one](https://www.americantransplantfoundation.org/about-transplant/facts-and-myths/).

In [33]:
import requests
url = 'https://www.bbc.com/news/world-us-canada-59944889'
url2 = 'https://www.americantransplantfoundation.org/about-transplant/facts-and-myths/'

question = 'How many people die every day in the US waiting for a transplant?'

# TODO. Impress me!